In [156]:
!pip install vrcpy
!pip install plyer

In [1]:
import vrcpy
import json
import itertools
from plyer import notification

In [2]:
with open("./setting.json", "r") as f:
    setting = json.load(f)

client = vrcpy.Client()
client.login(**setting)

In [ ]:
# name_list -> id_list

with open("name_list.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]

res = [client.api.call("/users", params={"search":name}) for name in lines]
res = list(itertools.chain.from_iterable([r["data"] for r in res]))
res = [{"id": r["id"], "username": r["username"], "displayName": r["displayName"]} for r in res]

s = [f'{r["id"]}: {r["displayName"]}/{r["username"]}' for r in res]
print("\n".join(s))

with open("id_list.txt", "w", encoding="utf-8") as f:
    for r in res:
        f.write(f'{r["id"]} # {r["displayName"]}/{r["username"]}\n')

In [4]:
# main function

with open("id_list.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

lines = [l.split("#")[0] for l in lines]
lines = [l.strip() for l in lines]
lines = [l for l in lines if len(l) > 0]

res = [client.fetch_user_by_id(l) for l in lines]
res = [r for r in res if hasattr(r, "state")]

online_friends = [r.displayName for r in res if r.state == "online"]

In [5]:
if (len(online_friends) > 0):
    s = ", ".join(online_friends)
    notification.notify(
        title = "フレンドがオンラインです",
        message=f'{s} is online now',
        app_name='VRC Online Friend Checker',
    )